# AlexNet Demo , Determine Cat and Dog.

In [1]:
import sys
sys.path.append('.')
import tensorflow as tf
# https://blog.csdn.net/weixin_44048809/article/details/105711356
import keras
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.utils import np_utils
from keras.optimizers import Adam
from model.AlexNet import AlexNet
import numpy as np
import utils
import cv2
from keras import backend as K
K.image_data_format() == 'channel_first'
import os 
IMG_PATH = os.path.join('.','data','image','train')
keras.__version__
cv2.__version__

'3.4.2'

In [ ]:
#predict
model = AlexNet()
model.load_weights(os.path.join('.','logs','last1.h5'))
img = cv2.imread(os.path.join('.','test2.jpg'))
img_RGB = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img_nor = img_RGB/255
img_nor = np.expand_dims(img_nor,axis=0)
print(img_nor.shape)
img_re = utils.resize_image(img_nor,(224,224))
print('the answer is :',utils.print_answer(np.argmax(model.predict(img_re))))
cv2.imshow('res',img)
cv2.waitKey(0)
cv2.destoryWindow('res')


(1, 374, 500, 3)
the answer is : 猫


In [ ]:
# train-file source 
def generate_arrays_from_file(lines,batch_size):
    # read length
    n = len(lines)
    i = 0
    while 1:
        X_train = []
        Y_train = []
        # get data @ batch_size
        for b in range(batch_size):
            if i == 0:
                np.random.shuffle(lines)
            name = lines[i].split(';')[0]
            # read image from file
            img = cv2.imread(os.path.join(IMG_PATH,name))
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img = img / 255
            X_train.append(img)
            Y_train.append(lines[i].split(';')[1])
            # restart a new period to read
            i = (i+1)%n
        #process image
        X_train = utils.resize_image(X_train,(224,224))
        X_train = X_train.reshape(-1,224,224,3)
        Y_train = np_utils.to_categorical(np.array(Y_train),num_classes = 2)
        yield(X_train,Y_train)
            

In [ ]:
log_dir = os.path.join('.','logs')

with open(os.path.join('.','data','dataset.txt'),'r') as f:
    lines = f.readlines()

# random to help train
np.random.seed(10101)
np.random.shuffle(lines)
np.random.seed(None)

# 90% train , 10& evaluate
num_val = int(len(lines)*0.1)
num_train = len(lines) - num_val

#model declare
model = AlexNet()

#save method , 3 epoch 1 time
checkpoint_period1 = ModelCheckpoint(
    os.path.join(log_dir,'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'),
    monitor = 'acc',
    save_weights_only=False,
    save_best_only = True,
    period = 3
)
    
#leraning rate reduce method, when acc not reduce three times, reduce the LR to continue train
reduce_lr = ReduceLROnPlateau(monitor='acc', factor=0.5, patience=3, verbose=1)
    
# when var_loss not reduce , can early stopping
early_stoppong = EarlyStopping(monitor='var_loss',
                              min_delta = 0,
                              patience = 10,
                              verbose = 1)

model.compile(loss = 'categorical_crossentropy',
             optimizer=Adam(lr=1e-3),
             metrics=['accuracy'])

batch_size = 128
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))

model.fit(generate_arrays_from_file(lines[:num_train],batch_size),
                   steps_per_epoch=max(1,num_train//batch_size),
                   validation_data = generate_arrays_from_file(lines[num_train:],batch_size),
                   epochs=50,
                   initial_epoch=0,
                   callbacks=[checkpoint_period1,reduce_lr]
                   )
model.save_weights(os.path.join(log_dir,'last2.h5'))
    
    